In [ ]:
import json
import os
import numpy as np
import SimpleITK as sitk
import csv

from library_dicom.dicom_processor.model.reader.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.reader.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.builder.mask_csv_builder.Mask import Mask 
from library_dicom.dicom_processor.model.builder.mask_csv_reader.MaskBuilder import MaskBuilder
from library_dicom.dicom_processor.tools.cleaning_dicom.folders import *
from library_dicom.dicom_processor.tools.visualization.create_mip import *
from library_dicom.dicom_processor.tools.pre_processing.threshold_mask import *

#### - Get dataset from  PARSE_FILTERED_DICOM_DATASET script

In [ ]:
#LIST_PATH from json
json_path = 'abs//path//json_file generated by parse_filtered_dicom_dataset.ipynb'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [ ]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

#### - Add CSV file for every study on the dataset (if available)

In [ ]:
csv_directory = 'abs//path//csv//directory'
list_csv = os.listdir(csv_directory)
list_csv_paths = []
for csv_ in list_csv : 
    list_csv_paths.append(os.path.join(csv_directory, csv_))
#in the csv name, has to be the patient_id in it
for patient in dataset : 
    #patient = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id ]
    patient_id = patient[-1]
    for csv_file in list_csv_paths: 
        if patient_id in csv_file: 
            patient.append(csv_file)

#### - Verify if every study has a csv file. If not, remove study from dataset 

In [ ]:
no_csv = []
for patient in dataset : 
    if len(data) != 8 : #7 if csv not available
        no_csv.append(data)
print("Number of study with no corresponding csv found :", len(no_csv))
try : 
    for data in no_csv : 
        dataset.remove(data)
except Exception as err : 
    print(err) 
print('Dataset cleaned')

In [ ]:
#SAVE DATASET AS JSON 
write_json_file('abs//path//directory', 'name of the json file', dataset)

#### - Remove remove_bi_file if it exists in every serie 

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

## Different scripts : 

#### - Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

In [ ]:
nifti_directory = 'abs//path//nifti//directory'
mip_directory = 'abs//path//mip//directory'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []
#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            serie_pt_objet.set_ExporType('suv')
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_builder_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_builder_objet.mask_array
            number_roi = mask_4D.shape[3]
            threshold = mask_builder_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                Mask(mask_4D, serie_pt_objet).export_nifti(filename_mask
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)
                
                #create mip for false mask and save
                #pet
                nifti_array = np.transpose(nifti_array,(2,0,1))
                angle_filename = mip_projection(nifti_array, angle=0, study_uid=study_uid, type='pet', cmap='Greys', vmin=0, vmax=7, directory_path=mip_directory)
                subliste.append(angle_filename)
                print('MIP PET')
               # mask
                mask_4D = np.transpose(mask_4D, (2,0,1,3))
                angle_filename_mask = mip_projection(mask_4D, angle=0, study_uid=study_uid, type='mask', cmap='Greys', vmin=0, vmax=0, directory_path=mip_directory)
                subliste.append(angle_filename_mask)
                print('MIP MASK')
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.set_ExportType('suv')
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_builder_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_builder_objet.mask_array
            number_roi = mask_4D.shape[3]
            threshold = mask_builder_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt), sul_value)
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                Mask(mask_4D, serie_pt_objet).export_nifti(filename_mask)
                print("EXPORT NIFTI MASK")

            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and save 
                #pet
                nifti_array = np.transpose(nifti_array,(2,0,1))
                angle_filename = mip_projection(nifti_array, angle=0, study_uid=study_uid, type='pet', cmap='Greys', vmin=0, vmax=7, directory_path=mip_directory)
                subliste.append(angle_filename)
                print('MIP PET')
               # mask
                mask_4D = np.transpose(mask_4D, (2,0,1,3))
                angle_filename_mask = mip_projection(mask_4D, angle=0, study_uid=study_uid, type='mask', cmap='Greys', vmin=0, vmax=0, directory_path=mip_directory)
                subliste.append(angle_filename_mask)
                print('MIP MASK ')
                path_mip.append(subliste)
               
    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

In [ ]:
#Save different results json file 
write_json_file(nifti_directory, 'false_mask_dataset', serie_false_mask)
write_json_file(nifti_directory, 'error_dataset', error_dataset)


#### - Generate PET, CT, MASK NIFTI without checking

In [ ]:

nifti_directory = 'path/nifti/directory'
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            serie_pt_objet.set_ExportType('suv')
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()

            mask_builder_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_builder_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            Mask(mask_4D, serie_pt_objet).export_nifti(filename_mask)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.set_ExportType('suv')
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()

            mask_builder_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_builder_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            Mask(mask_4D, serie_pt_objet).export_nifti(filename_mask)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))
write_json_file(nifti_directory, 'error_dataset', error_dataset)

#### - Generate PET & CT NIFTI only

In [ ]:
nifti_directory  = 'path/nifti_/directory'
error = []

for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                serie_pt_objet.set_ExportType('suv')
                serie_pt_objet.get_instances_ordered()
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2])
                serie_pt_objet.set_ExportType('suv')
                serie_pt_objet.get_instances_ordered() 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)                                                                                                                                                                                                                              

In [ ]:
print("number of error : ", len(error_dataset))
write_json_file(nifti_directory, 'error_dataset', error_dataset)